In [69]:
import numpy as np
import torch
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping 
import cmath
from torch import nn
import random
from torch.utils import data

In [2]:
class generator_t:
    def __init__(self,df,mu = 0, sigma = 1,samplesize = 1000, h = 0.01):
        self.df = df
        self.mu = mu
        self.sigma = sigma
        self.samplesize = samplesize
        self.frequency = h
    @staticmethod
    def cumulate(delta):
        data = torch.zeros(np.shape(delta))
        n = np.shape(delta)[0]
        for i in range(n-1):
            data[i+1] = data[i]+delta[i]
        return np.array(data)
    
    def generate_standard_t(self):
        d0 = np.random.standard_t(self.df, (self.samplesize,1))
        d1 = self.cumulate(d0)
        delta = torch.tensor(d0)
        data = torch.tensor(d1)
        delta.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
        return delta,data

### Characteristic Function

In [55]:
class Characteristic_function:
    def __init__(self,data):
        self.data = data   
    def ecf(self,t):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            return c,s                
        else:    
            m = len(t)
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            R = torch.zeros((m,2))
            for j in range(m):
                for i in range(n):
                    C[i] = math.cos(t[j]*self.data[i])
                    S[i] = math.sin(t[j]*self.data[i])
                s = float(torch.mean(S))
                c = float(torch.mean(C))
                R[j,0] = c
                R[j,1] = s
        return R
    def ecf_h(self,t,h):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            f = complex(c,s)
            q,w = cmath.polar(f)
            q = q**h
            w = w*h
            c = math.cos(w)*q
            s = math.sin(w)*q
            return c,s                
        else:    
            m = len(t)
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            R = torch.zeros((m,2))
            for j in range(m):
                for i in range(n):
                    C[i] = math.cos(t[j]*self.data[i])
                    S[i] = math.sin(t[j]*self.data[i])
                s = torch.mean(S)
                c = torch.mean(C)
                f = complex(c,s)
                q,w = cmath.polar(f)
                q = q**h
                w = w*h
                c = math.cos(w)*q
                s = math.sin(w)*q
                R[j,0] = c
                R[j,1] = s
        return R


In [109]:
class Characteristic_function:
    def __init__(self,data):
        self.data = data   
    def ecf(self,t):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            return c,s                
        else:    
            m = len(t)
            C = []
            S = []
            R = []
            for j in range(m):
                for i in range(n):
                    C.append(math.cos(t[j]*self.data[i]))
                    S.append(math.sin(t[j]*self.data[i]))
                S = torch.tensor(S,dtype = torch.float64)
                C = torch.tensor(C,dtype = torch.float64)
                s = float(torch.mean(S))
                c = float(torch.mean(C))
                R.append(c)
                R.append(s)
                R = torch.tensor(R,dtype = torch.float64)
        return R
    def ecf_h(self,t,h):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            f = complex(c,s)
            q,w = cmath.polar(f)
            q = q**h
            w = w*h
            c = math.cos(w)*q
            s = math.sin(w)*q
            return c,s                
        else:    
            m = len(t)
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            R = torch.zeros((m,2))
            for j in range(m):
                for i in range(n):
                    C[i] = math.cos(t[j]*self.data[i])
                    S[i] = math.sin(t[j]*self.data[i])
                s = torch.mean(S)
                c = torch.mean(C)
                f = complex(c,s)
                q,w = cmath.polar(f)
                q = q**h
                w = w*h
                c = math.cos(w)*q
                s = math.sin(w)*q
                R[j,0] = c
                R[j,1] = s
        return R


In [110]:
a = [2,1]
a.append(1)
a = torch.tensor(a,dtype = torch.float64)
torch.mean(a)

tensor(1.3333, dtype=torch.float64)

In [111]:
num_feature = 20
range_feature = 10
num_t = 10
range_t = 5
num_labels = num_t
num_sample = 15
total = num_sample*num_feature
t = torch.linspace(-range_t,range_t,num_t)
x = torch.linspace(-range_feature,range_feature,num_feature)

In [112]:
#实例化(V =3)
Gen3 = generator_t(3,samplesize =total)

delta,path = Gen3.generate_standard_t()
Chf = Characteristic_function(delta)

In [113]:
a = []
for i in range(num_sample):
    delta,path = Gen3.generate_standard_t()
    Chf = Characteristic_function(delta)
    labels = Chf.ecf(t).reshape((-1,1))
    a.append([t,labels])


AttributeError: 'Tensor' object has no attribute 'append'

In [80]:
df = pd.DataFrame(a)
features = df[0]
labels = df[1]
df[1]

0     [[tensor(0.0101)], [tensor(0.0160)], [tensor(0...
1     [[tensor(-0.0109)], [tensor(0.0002)], [tensor(...
2     [[tensor(0.0036)], [tensor(0.0090)], [tensor(0...
3     [[tensor(-0.0158)], [tensor(0.0047)], [tensor(...
4     [[tensor(0.0123)], [tensor(0.0004)], [tensor(0...
                            ...                        
95    [[tensor(0.0041)], [tensor(0.0045)], [tensor(8...
96    [[tensor(-0.0022)], [tensor(-0.0007)], [tensor...
97    [[tensor(0.0026)], [tensor(-0.0081)], [tensor(...
98    [[tensor(0.0055)], [tensor(-0.0086)], [tensor(...
99    [[tensor(-0.0067)], [tensor(0.0181)], [tensor(...
Name: 1, Length: 100, dtype: object

In [70]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [77]:
D_loader = data.DataLoader(df, 10, shuffle=False)

In [79]:
next(iter(D_loader)) 

KeyError: 2

In [72]:
batch_size = 10
data_iter = load_array(df, batch_size)

AttributeError: 'int' object has no attribute 'size'

In [37]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [38]:
for X, y in data_iter(10, features, labels):
    print(X, '\n', y)
    break

KeyError: tensor([47, 63, 15, 57, 54, 90, 59, 62, 37,  5])

In [26]:
hidden_2, hidden_3 = 256, 256

W1 = nn.Parameter(torch.randn(
    num_feature, hidden_2, requires_grad=True,dtype = torch.complex64) * 0.01)
b1 = nn.Parameter(torch.zeros(hidden_2, requires_grad=True,dtype = torch.complex64))

W2 = nn.Parameter(torch.randn(
    hidden_2, hidden_3, requires_grad=True,dtype = torch.complex64) * 0.01)
b2 = nn.Parameter(torch.zeros(hidden_3, requires_grad=True,dtype = torch.complex64))

W3 = nn.Parameter(torch.randn(
    hidden_3, num_feature, requires_grad=True,dtype = torch.complex64) * 0.01)
b3 = nn.Parameter(torch.zeros(num_feature, requires_grad=True,dtype = torch.complex64))


params = [W1, b1, W2, b2, W3, b3]